In [1]:
from dolfin import *
import mshr
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML 
import time

In [2]:
def stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f, folder_name):     
    dt = T / num_steps # time step size
    t = dt
    mesh_f_s = []
    
    
    
    #input mesh choose
    if mesh_f == 0:
        mesh = Mesh('stenosis_f0.6.xml')
        boundary = MeshFunction('size_t', mesh,'stenosis_f0.6_facet_region.xml')
        mesh_f_s = 'R' #regular
    else:
        mesh = Mesh('stenosis_f0.6_fine.xml')
        boundary = MeshFunction('size_t', mesh,'stenosis_f0.6_fine_facet_region.xml')
        mesh_f_s = 'F' #fine
    h = CellDiameter(mesh)
    
    
    
    # Build function space
    P2 = VectorElement("P", mesh.ufl_cell(), element_n)
    P1 = FiniteElement("P", mesh.ufl_cell(), 1)
    TH = P2 * P1
    W = FunctionSpace(mesh, TH)
    u_par = Constant((0,0))
    
    
    
    #boundary
    ds = Measure('ds', domain=mesh, subdomain_data=boundary)
    n_v = FacetNormal(mesh)
    boundary_conditions = {1: {'Dirichlet': u_par},   #inflow
                           #2: {'Dirichlet': Constant(0)},   #outflow
                           #3: {'Dirichlet': Constant(0)},   #symmetry
                           4: {'Dirichlet': Constant((0.0, 0.0))}}   #no-slip
        # Collect Dirichlet conditions
    bc = DirichletBC(W.sub(0), boundary_conditions[1]['Dirichlet'], boundary, 1)
    
    

    # Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    u0 = project(Constant((0, 0)), VectorFunctionSpace(mesh, "P", 4))
    p0 = project(Constant(0), FunctionSpace(mesh, "P", 4))
    
    
    
    #f constant
    f = Constant((0, 0))
    f0 = Constant((0, 0))

    
    
    #file work
    folder =  folder_name + '/cn_' +str(conv_f)+ '-bc_' +str(bcflow_f)+ '-te_' +str(temam_f)+ '-su_' +str(supg_f)+ '-co_' +str(combi_f) + '_'+mesh_f_s+ '-dt_' + str(dt) + '-e_' + str(epsilon) + '-th_' + str(theta)  
    u_file = XDMFFile(folder + '/u.xdmf')
    p_file = XDMFFile(folder + '/p.xdmf')
    u_file.parameters['rewrite_function_mesh'] = False
    p_file.parameters['rewrite_function_mesh'] = False
    u_file.parameters["flush_output"] = True
    p_file.parameters["flush_output"] = True
    
    
    
    # Time-stepping
    proceent = 0
    t_arr = np.zeros(num_steps)
    
    
    
    #constant
    f = Constant((0, 0))
    f0 = Constant((0, 0))
    
    
    
    x, y = sym.symbols('x[0], x[1]')  
    u_par = (1.5 *Re *mu/2/rho/R)*(1 - (y/R)**2)
    u_D = sym.simplify(u_par)
    u_par_c = Expression((sym.printing.ccode(u_D), 0), degree=5)
    
    for n in range(num_steps):
        
        
        #progress
        if progress_f == 1:
            if int(n/(num_steps-1)*100) >= proceent:
                print("% =", int(n/(num_steps-1) * 100), ', time mplsd =', int((time.time()-t_start)))
                proceent+=proceents
        t_arr[n] = t
        
        
        #calculate u_bulk
        u_par = u_par_c*np.sin(np.pi*t/T)
        u_par = project(u_par, VectorFunctionSpace(mesh, "P", 4))
        bc_4 = DirichletBC(W.sub(0), u_par, boundary, 4)
        
        
        
        #boundary calculation
        bcs = []
        bcs.append(bc)
        bcs.append(bc_4)
        bc_3 = - inner(p0*n_v, v)*ds
        
        
        
        #Form parts calculation 
        a1 = (mu*inner(grad(u0), grad(v)) + div(v)*p0 + mu*q*div(u0))*dx(domain = mesh)
        L1 = inner(f0, v)*dx(domain = mesh)
        a2 = (mu*inner(grad(u), grad(v)) + div(v)*p + mu*q*div(u))*dx(domain = mesh) 
        L2 = inner(f, v)*dx(domain = mesh)
        
        
        
        
        #correction terms
        correction_t_r = temam_t_r = epsilon_t_r = conv_t_r = bcflow_t_r = supg_t_r = combi_t_r = temam_t_l = epsilon_t_l = conv_t_l = bcflow_t_l = supg_t_l = combi_t_l = 0
        
        
        if bcflow_f == 1:
            ala = inner(u0,n_v)
            #bcflow_t = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner((theta*u - (1-theta)*u0), v) * dx(domain = mesh) 
            bcflow_t_l = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner((theta*u), v) * ds
            bcflow_t_r = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner(((1-theta)*u0), v) * ds
        else:
            bcflow_t_l = bcflow_t_r = 0
        
        
        
        if temam_f == 1:
            #temam_t = temam_f * rho/2 * inner(dot(grad(u0),(theta*u - (1-theta)*u0)), v)*dx(domain = mesh) 
            temam_t_l = temam_f * rho/2 * inner(dot(grad(u0),(theta*u)), v)*dx(domain = mesh)
            temam_t_r = temam_f * rho/2 * inner(dot(grad(u0),((1-theta)*u0)), v)*dx(domain = mesh)                                   
        else:
            temam_f_t_l = temam_f_t_r = 0
        
        
        
        if epsilon == 1:
            #epsilon_t =  epsilon *h*h/mu*inner(grad(q), grad((theta*p - (1-theta)*p0)))*dx(domain = mesh)
            epsilon_t_l =  epsilon *h*h/mu*inner(grad(q), grad(theta*p))*dx(domain = mesh) 
            epsilon_t_r =  epsilon *h*h/mu*inner(grad(q), (1-theta) * grad(p0))*dx(domain = mesh)
        else:
            epsilon_t_l = epsilon_t_r = 0
        
        
        
        if conv_f == 1:                              
            #conv_t =  conv_f * rho*inner(dot(u0, nabla_grad((theta*u - (1-theta)*u0))), v) * dx(domain = mesh) 
            conv_t_l =  conv_f * rho*inner(dot(u0, nabla_grad((theta*u))), v) * dx(domain = mesh)
            conv_t_r =  conv_f * rho*inner(dot(u0, (1-theta)*nabla_grad((u0))), v) * dx(domain = mesh)                                    
        else:
            conv_t_l = conv_t_r = 0
        
        
        
        if supg_f == 1:                              
            supg_t_ =  supg_f * (4/dt/dt + 4*dot(u0,u0)/h/h + (12*mu/rho/h/h)**2)**(-0.5)
            #supg_t =  supg_t_ * rho * inner(dot(u0, nabla_grad((theta*u - (1-theta)*u0))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)
            supg_t_l =  supg_t_ * rho * inner(dot(u0, nabla_grad((theta*u))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)                                     
            supg_t_r =  supg_t_ * rho * inner(dot(u0, (1-theta)*nabla_grad((u0))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)                                                                    
        else:
            supg_t_l = supg_t_r = 0
        
        
        
        if combi_f == 1:
            supg_t_ =  supg_f * (4/dt/dt + 4*dot(u0,u0)/h/h + (12*mu/rho/h/h)**2)**(-0.5)
            #combi_t =  combi_f * supg_t_ * inner((rho * dot(u0, nabla_grad((theta*u - (1-theta)*u0))) + grad((theta*p - (1-theta)*p0))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh)  
            combi_t_l =  combi_f * supg_t_ * inner((rho * dot(u0, nabla_grad((theta*u))) + grad((theta*p))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh) 
            combi_t_r =  combi_f * supg_t_ * inner((rho * dot(u0, (1-theta)*nabla_grad((u0))) + (1-theta)*grad((p0))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh) 
        else:
            combi_t_l = combi_t_r = 0
        
        
                                            
        #Form calculation 
        correction_t_r = temam_t_r + epsilon_t_r + conv_t_r + bcflow_t_r + supg_t_r + combi_t_r +bc_3
        correction_t_l = temam_t_l + epsilon_t_l + conv_t_l + bcflow_t_l + supg_t_l + combi_t_l
        
        
        a = inner(u,v) * dx + (theta) * (a2) + correction_t_l
        L =  (1 - theta) * (a1 - L1) - (theta) * L2 - inner(u0,v)*dx + correction_t_r
        U = Function(W)
        solve(a == L, U, bcs)
        u0, p0 = U.split()
        
        
        
        #file work
        u0.rename("v0", "velocity")
        p0.rename("p0", "pressure")
        u_file.write(u0, t)
        p_file.write(p0, t)
        
        
        
        #time step apply
        t += dt
    
    
    
    #file close
    u_file.close()
    p_file.close()
    if plot_f == 1:
        plt.plot(t_arr,vc_arr)

2.5 Convection stabilization (1 point)
Include additionally a SUPG stabilization
$$
\int_{\Omega} \delta\left(\boldsymbol{u}^{k}\right)\left(\rho \boldsymbol{u}^{k} \cdot \nabla\right) \boldsymbol{u}^{k+\theta} \cdot\left(\rho \boldsymbol{u}^{k} \cdot \nabla\right) \boldsymbol{v}
$$
Use the following easy-to-implement simplification of the optimal formula used in Lab 3 for the stabilization parameter:
$$
\delta\left(\boldsymbol{u}^{k}\right)=\left(\frac{4}{\tau^{2}}+4 \frac{\left\|\boldsymbol{u}^{k}\right\|^{2}}{h^{2}}+\left(\frac{12 \mu}{\rho h^{2}}\right)^{2}\right)^{-\frac{1}{2}}
$$
$\tau$ is the time step size, $\|\boldsymbol{u}\|^{2}$ is the square of the element-wise $L^{2}$ norm and can be implemented as $\operatorname{dot}(u, u)$.

Assignment ( 1 point) Compare the solution for $R e=25000$ (yes, twenty five tousend!) and $\varepsilon=0.0001$ and $\mathbb{P}_{1} / \mathbb{P}_{1}$, in the following four cases: with and without SUPG, for both fine and coarse meshes. Explain how the solution changes (numerical wiggles, physical vortices) when changing the mesh and adding SUPG.

without SUPG, coarse mesh

In [3]:
folder_name = '6.2.5/coarse/nosupg'
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 0.5
R = 1
element_n = 1 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 25000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f,folder_name)

% = 0 , time mplsd = 3
% = 10 , time mplsd = 39
% = 20 , time mplsd = 74
% = 30 , time mplsd = 109
% = 40 , time mplsd = 145
% = 50 , time mplsd = 181
% = 60 , time mplsd = 215
% = 70 , time mplsd = 250
% = 80 , time mplsd = 285
% = 90 , time mplsd = 321
% = 100 , time mplsd = 356


with SUPG, coarse mesh

In [4]:
folder_name = '6.2.5/coarse/supg'
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 0.5
R = 1
element_n = 1 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 1
combi_f = 0


plot_f = 0

Re = 25000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f,folder_name)

% = 0 , time mplsd = 1
Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
% = 10 , time mplsd = 41
% = 20 , time mplsd = 77
% = 30 , time mplsd = 114
% = 40 , time mplsd = 152
% = 50 , time mplsd = 188
% = 60 , time mplsd = 225
% = 70 , time mplsd = 264
% = 80 , time mplsd = 303
% = 90 , time mplsd = 341
% = 100 , time mplsd = 377


without SUPG, fine mesh

In [3]:
folder_name = '6.2.5/fine/nosupg'
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 0.5
R = 1
element_n = 1 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 1 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 25000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f,folder_name)

% = 0 , time mplsd = 5
% = 10 , time mplsd = 159
% = 20 , time mplsd = 313
% = 30 , time mplsd = 467
% = 40 , time mplsd = 626
% = 50 , time mplsd = 781
% = 60 , time mplsd = 934
% = 70 , time mplsd = 1089
% = 80 , time mplsd = 1243
% = 90 , time mplsd = 1399
% = 100 , time mplsd = 1554


with SUPG, fine mesh

In [4]:
folder_name = '6.2.5/fine/supg'
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 0.5
R = 1
element_n = 1 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 1 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 1
combi_f = 0


plot_f = 0

Re = 25000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f,folder_name)

% = 0 , time mplsd = 4
% = 10 , time mplsd = 164
% = 20 , time mplsd = 328
% = 30 , time mplsd = 489
% = 40 , time mplsd = 650
% = 50 , time mplsd = 812
% = 60 , time mplsd = 983
% = 70 , time mplsd = 1191
% = 80 , time mplsd = 1406
% = 90 , time mplsd = 1608
% = 100 , time mplsd = 1775
